## Step 1: Import Libraries and Load Dataset

In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# Load dataset
df = pd.read_csv('bank-additional-full.csv', sep=';')
df.columns = df.columns.str.strip()
df.head(2)


## Step 2: Data Exploration and Visualization

In [ ]:

print(df.columns.tolist())

sns.countplot(data=df, x='y')
plt.title('Target Class Distribution')
plt.savefig("target_distribution.png")
plt.show()

plt.figure(figsize=(12,5))
sns.countplot(data=df, x='job', hue='y')
plt.xticks(rotation=45)
plt.title('Job Type vs Subscription')
plt.savefig("job_vs_subscription.png")
plt.show()

plt.figure(figsize=(12,5))
sns.countplot(data=df, x='education', hue='y')
plt.xticks(rotation=45)
plt.title('Education vs Subscription')
plt.savefig("education_vs_subscription.png")
plt.show()

plt.figure(figsize=(12,8))
corr = df.select_dtypes(include='number').corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.savefig("correlation_matrix.png")
plt.show()


## Step 3: Data Preprocessing and Feature Engineering

In [ ]:

df.replace('unknown', np.nan, inplace=True)
df.dropna(inplace=True)

binary_cols = ['default', 'housing', 'loan', 'y']
for col in binary_cols:
    df[col] = df[col].map({'yes':1, 'no':0})

cat_cols = df.select_dtypes(include='object').columns.tolist()
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

scaler = StandardScaler()
num_cols = df.select_dtypes(include='number').columns.tolist()
num_cols.remove('y')
df[num_cols] = scaler.fit_transform(df[num_cols])


## Step 4: Model Building

In [ ]:

X = df.drop('y', axis=1)
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

dt = DecisionTreeClassifier(random_state=42)
params = {'max_depth': [4, 6, 8, 10],
          'min_samples_split': [2, 5, 10],
          'criterion': ['gini', 'entropy']}

grid = GridSearchCV(dt, param_grid=params, cv=5)
grid.fit(X_train, y_train)
best_dt = grid.best_estimator_


## Step 5: Model Evaluation

In [ ]:

y_pred_log = logreg.predict(X_test)
y_pred_dt = best_dt.predict(X_test)

print("Logistic Regression:")
print(classification_report(y_test, y_pred_log))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_log))
log_roc_auc = roc_auc_score(y_test, y_pred_log)
print("ROC AUC:", log_roc_auc)

print("\nDecision Tree:")
print(classification_report(y_test, y_pred_dt))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_dt))
tree_roc_auc = roc_auc_score(y_test, y_pred_dt)
print("ROC AUC:", tree_roc_auc)


## Step 6: Final Summary

In [ ]:

print(f"\nLogistic Regression ROC AUC: {log_roc_auc:.4f}")
print(f"Decision Tree ROC AUC: {tree_roc_auc:.4f}")

if log_roc_auc > tree_roc_auc:
    print("\n✅ Logistic Regression performs better based on ROC AUC.")
else:
    print("\n✅ Decision Tree performs better based on ROC AUC.")

print("\n📁 All plots saved as PNG files in your working directory.")
